# Robust Optimization for Genetic Selection

This is a working notebook, looking at how the quadratic optimization problem (QP) which arises in the context of robust genetic selection can be solved with Python (tested with 3.10 specifically). There are some standard packages which this depends on, imported below.

In [1]:
import numpy as np                  # defines matrix structures
from numpy import linalg as nla     # access Numpy's NLA functions
from qpsolvers import solve_qp      # used for quadratic optimization
from time import perf_counter       # fine grained timing 
import gurobipy as gp               # Gurobi optimization interface (1)
from gurobipy import GRB            # Gurobi optimization interface (2)

Utility functions and output settings used in this notebook are defined in the two cells below.

In [2]:
def printTime(task, tic, toc):
    """Quick function for nicely printing a time to 5 s.f."""
    print(f"{task} in {toc - tic:0.5f} seconds\n")


def printMatrix(matrix, description="ans =", precision=3):
    """Quick function for nicely printing a matrix"""
    print(f"{description}\n", np.round(matrix, precision))

In [3]:
# want to round rather than truncate when printing
np.set_printoptions(threshold=np.inf)

## Reading Pedigrees

We start by reviewing the basic functions needed for handling genetics input data, describing the relationship structure of a cohort of organisms.

These structures, sometimes referred to as pedigrees, are commonly stored in `*.ped` data files. The files are essentially a CSV format with three columns $i$, $p$, and $q$, where $i$ is the index of the organism, and $p$ and $q$ are indices of $i$'s parents. Unknown parents (_i.e._ where the parent of an organism within the cohort is from outside the cohort) are represented by a zero. Note that it doesn't matter whether each of $p$ and $q$ is a sire or dam, but the labelling must be such that $p < q$.

We read these files into a dictionary using `readPed` below.

In [4]:
def readPed(filename):
    """
    Function for reading *.ped files to a dictionary. Takes the file name
    as a string input and returns the pedigree structure as a dictionary.
    """
    with open(filename, "r") as file:
        # first line of *.ped lists the headers; skip
        file.readline()
        # create a list of int lists from each line (dropping optional labels)
        data = [[int(x) for x in line.split(",")[0:3]] for line in file]
    # convert this list of lists into a dictionary
    ped = {entry[0]: entry[1:3] for entry in data}
    return(ped)

Once we have read the pedigree file we can construct Wright's Numerator Relationship Matrix (WNRM), a matrix $A$ which encodes the relationship between organisms $i$ and $j$ as $A_{ij}$. This defined as
$$
    A_{(i, j)} = A_{(j, i)} = \frac{1}{2}\left( A_{(j,p)} + A_{(j,q)} \right),~\forall i = 1, \ldots, m,~\forall j=1,\ldots,i-1
$$
for the off diagonal entries and
$$
    A_{(i,i)} = 1 + \frac{1}{2}A_{(p,q)}
$$
for the diagonal entries.

In [5]:
def makeA(pedigree):
    """
    Construct Wright's Numerator Relationship Matrix from a given pedigree
    structure. Takes the pedigree as a dictionary input and returns the
    matrix as output.
    """
    m = len(pedigree)
    A = np.zeros((m, m))

    # iterate over rows
    for i in range(0, m):
        # save parent indexes: pedigrees indexed from 1, Python from 0
        p = pedigree[i+1][0]-1
        q = pedigree[i+1][1]-1
        # iterate over columns sub-diagonal
        for j in range(0, i):
            # calculate sub-diagonal entries
            A[i, j] = 0.5*(A[j, p] + A[j, q])
            # populate sup-diagonal (symmetric)
            A[j, i] = A[i, j]
        # calculate diagonal entries
        A[i, i] = 1 + 0.5*A[p, q]

    return(A)

For our work we also need the inverse of this matrix, $A^{-1}$. While we could compute this explicitly or using a decomposition, it's more efficient to use shortcuts highlighted by Henderson and Quass. Given $A$ is a positive definite matrix, it has a Cholesky factorisation $A = L^{T}L$. We compute this factor using `makeL` below.

In [6]:
def makeL(pedigree):
    """
    Construct the Cholesky factor L of Wright's Numerator Relationship Matrix
    from a given pedigree structure. Takes the pedigree as a dictionary input
    and returns the matrix L (in A = L'L) as output.
    """
    m = len(pedigree)
    L = np.zeros((m, m))

    # iterate over rows
    for i in range(0, m):
        # save parent indexes: pedigrees indexed from 1, Python from 0
        p = pedigree[i+1][0]-1
        q = pedigree[i+1][1]-1

        # case where both parents are known; p < q bny *.ped convention
        if p >= 0 and q >= 0:
            for j in range(0, p+1):
                L[i, j] = 0.5*(L[p, j] + L[q, j])
            for j in range(p+1, q+1):
                L[i, j] = 0.5*L[q, j]
            # and L[i,j] = 0 for j = (q+1):i

            # compute the diagonal
            s = 1
            for j in range(0, p+1):
                s += 0.5*L[p, j]*L[q, j]
            for j in range(0, q+1):
                s -= L[i, j]**2
            L[i, i] = s**0.5

        # case where one parent is known; p by *.ped convention
        elif p >= 0:  # and q = 0
            for j in range(0, p+1):
                L[i, j] = 0.5*L[p, j]
            # and L[i,j] = 0 for j = (q+1):i

            # compute the diagonal
            s = 1
            for j in range(0, p+1):
                s -= L[i, j]**2
            L[i, i] = s**0.5

        else:
            for j in range(0, i):
                L[i, j] = 0
            L[i, i] = 1

    return(L)

Using this $L$, and observing also that $A = L^{T}L = T^{T}DDT$ (where $T$ is a triangular matrix and $D$ a diagonal matrix), we further compute the inverse of $D^2$ using `make_invD2`.

In [7]:
def make_invD2(pedigree):
    """
    Construct the inverse of the D^2 factor from the Henderson (1976)
    decomposition of a WNRM. Takes the pedigree as a dictionary input
    and returns the inverse of matrix D^2 (in A = L'L = T'DDT) as output.
    """
    m = len(pedigree)
    L = makeL(pedigree)
    invD2 = np.zeros((m, m))  # TODO find a way to store diagonal matrices

    # iterate over rows
    for i in range(0, m):
        invD2[i, i] = 1/(L[i, i]**2)

    return(invD2)

Finally, we use $D^{-1}$ and exploit the structure of the pedigree (using shortcuts highlighted by Henderson) to compute $A^{-1}$ directly. The algorithm for this is given in `make_invA` below.

In [8]:
def make_invA(pedigree):
    """
    Compute the inverse of A using a shortcut which exploits
    of its T and D decomposition, detailed in Henderson (1976).
    Takes the pedigree as a dictionary input and returns the
    inverse as matrix output.
    """
    m = len(pedigree)
    B = make_invD2(pedigree)
    invA = B

    for i in range(0, m):
        # label parents p & q
        p = pedigree[i+1][0]-1
        q = pedigree[i+1][1]-1

        # case where both both parents are known
        if p >= 0 and q >= 0:
            x = -0.5*B[i, i]
            y = 0.25*B[i, i]
            invA[p, i] += x
            invA[i, p] += x
            invA[q, i] += x
            invA[i, q] += x
            invA[p, p] += y
            invA[p, q] += y
            invA[q, p] += y
            invA[q, q] += y

        # case where one parent is known; p by *.ped convention
        elif p >= 0:
            x = -0.5*B[i, i]
            invA[p, i] += x
            invA[i, p] += x
            invA[p, p] += 0.25*B[i, i]

    return(invA)


As an example, consider the following pedigree data from Henderson (1976), _A Simple Method for Computing the Inverse of a Numerator Relationship Matrix Used in Prediction of Breeding Values_ in Biometrics 32:1, pg. 69-83 (Table 1).

|$i$|$p$|$q$|
|:-:|:-:|:-:|
| i | p | q |
| 1 | 0 | 0 |
| 2 | 0 | 0 |
| 3 | 1 | 0 |
| 4 | 1 | 2 |
| 5 | 3 | 4 |
| 6 | 1 | 4 |
| 7 | 5 | 6 |

<!-- TODO add a split here, where on the left we have the table and on the right we have a graphical visualization of the pedigree tree -->


We can use the functions defined in this section to form $A$ and $A^{-1}$, and compare the inverse formed to the one formed using NumPy's standard Numerical Linear Algebra tools.  

In [9]:
# constructing the pedigree tree
ped = readPed("example2.ped")

# construction of A
tic = perf_counter()
A = makeA(ped)
toc = perf_counter()
printMatrix(A, "A =")
printTime("Constructed A", tic, toc)

# calculating inverse of A
tic = perf_counter()
numpy_inverse = np.linalg.inv(A)
toc = perf_counter()
printMatrix(numpy_inverse, "A^-1 =\t(from numpy)")
printTime("Numpy inversion", tic, toc)

# constructing inverse of A
tic = perf_counter()
hendo_inverse = make_invA(ped)
toc = perf_counter()
printMatrix(hendo_inverse, "A^-1 =\t(from pedigree)")
printTime("Henderson inversion", tic, toc)


A =
 [[1.    0.    0.5   0.5   0.5   0.75  0.625]
 [0.    1.    0.    0.5   0.25  0.25  0.25 ]
 [0.5   0.    1.    0.25  0.625 0.375 0.5  ]
 [0.5   0.5   0.25  1.    0.625 0.75  0.688]
 [0.5   0.25  0.625 0.625 1.125 0.562 0.844]
 [0.75  0.25  0.375 0.75  0.562 1.25  0.906]
 [0.625 0.25  0.5   0.688 0.844 0.906 1.281]]
Constructed A in 0.00011 seconds

A^-1 =	(from numpy)
 [[ 2.333  0.5   -0.667 -0.5    0.    -1.     0.   ]
 [ 0.5    1.5    0.    -1.     0.     0.     0.   ]
 [-0.667  0.     1.833  0.5   -1.     0.     0.   ]
 [-0.5   -1.     0.5    3.    -1.    -1.     0.   ]
 [ 0.     0.    -1.    -1.     2.615  0.615 -1.231]
 [-1.     0.     0.    -1.     0.615  2.615 -1.231]
 [ 0.     0.     0.     0.    -1.231 -1.231  2.462]]
Numpy inversion in 0.00093 seconds

A^-1 =	(from pedigree)
 [[ 2.333  0.5   -0.667 -0.5    0.    -1.     0.   ]
 [ 0.5    1.5    0.    -1.     0.     0.     0.   ]
 [-0.667  0.     1.833  0.5   -1.     0.     0.   ]
 [-0.5   -1.     0.5    3.    -1.    -1.   

From external testing we know that $A$ is 91.84\% dense while $A^{-1}$ is only 65.31\% dense, meaning the inverse is relatively sparse. We also have that $\lambda_{\max}(A) = 4.277$ while $\lambda_{\min}(A) = 0.183$, giving that $\kappa(A) = 23.351$.

## Exploring Toy Problems

Now we have worked out how to read in the relevant data, where this arises in optimization problems. In the context of genetic selection, we want to maximize selection of genetic merit (a measure of desirable traits) while minimizing risks due to inbreeding. This is formed mathematically as
$$
    \min_w \frac{1}{2}w^{T}\Sigma w - \lambda w^{T}\mu\ \text{ subject to }\ w_{\mathcal{S}}^{T}e_{\mathcal{S}}^{} = \frac{1}{2},\ w_{\mathcal{D}}^{T}e_{\mathcal{D}}^{} = \frac{1}{2},\ l\leq w\leq u,
$$
where $w$ is the vector of proportional contributions, $\Sigma$ is a matrix encoding risk, $\mu$ is a vector encoding returns, $l$ encodes lower bounds on contributions, $u$ encodes upper bounds on contributions, $\mathcal{S}$ is an index set of candidates who are sires, and $\mathcal{D}$ is an index set of candidates who are dams.

In this representation of the problem, $\lambda$ is a control variable which balances how we trade of between risk and return. Each value of $\lambda$ will give a different solution on the critical frontier of the problem. 

### Standard Optimization

We will start by looking how this problem might be solving using Python's [qpsolvers](https://qpsolvers.github.io/qpsolvers/index.html) library. Since it takes problems of the form
$$
    \min_x \frac{1}{2} x^T A x + q^T x\ \text{ subject to }\ Gx\leq h,\ Mx = m,\ l\leq x\leq u,
$$
we will need to do a very slight rearrangement of the problem to incorporate our two sum-to-half constraints within a single equality constraint. We also do not use the $Gx\leq h$ constraint in our problem.

We observe that the two vector constraints
$$
    w_{\mathcal{S}}^{T}e_{\mathcal{S}}^{} = \frac{1}{2},\ w_{\mathcal{D}}^{T}e_{\mathcal{D}}^{} = \frac{1}{2},
$$
are equivalent to the single matrix constraint
$$
    Mw := \begin{bmatrix}
        \mathbb{I}\lbrace 1\in\mathcal{S}\rbrace & \mathbb{I}\lbrace 2\in\mathcal{S}\rbrace & \cdots & \mathbb{I}\lbrace n\in\mathcal{S}\rbrace \\
        \mathbb{I}\lbrace 1\in\mathcal{D}\rbrace & \mathbb{I}\lbrace 2\in\mathcal{D}\rbrace & \cdots & \mathbb{I}\lbrace n\in\mathcal{D}\rbrace \end{bmatrix}w = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},
$$
where $\mathbb{I}\lbrace i\in\mathcal{I}\rbrace$ is an indicator function denoting whether index $i$ is in the set of indices $\mathcal{I}$.

### Toy Standard Example ($n=3$)

Lets see how this works in an example. Consider the problem where
$$
    \mu = \begin{bmatrix} 1 \\ 5 \\ 2, \end{bmatrix}\quad
    \Sigma = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 5 & 0 \\ 0 & 0 & 3 \end{bmatrix}, \quad
    \mathcal{S} = \lbrace 1 \rbrace, \quad
    \mathcal{D} = \lbrace 2, 3 \rbrace, \quad
    l = {\bf 0}, \quad
    u = {\bf 1}.
$$
We define these variables in Python using the following code.

In [10]:
# KEY PROBLEM VARIABLES
problem_size = 3
expected_breeding_values = np.array([
    1.0,
    5.0,
    2.0
])
relationship_matrix = np.array([
    [1, 0, 0],
    [0, 5, 0],
    [0, 0, 3]
])
sire_indices = [0]
dam_indices  = [1,2]
lower_bound = np.full((problem_size, 1), 0.0)
upper_bound = np.full((problem_size, 1), 1.0)

We have the additional variables which need setting up so that the problem works in `qpsolvers`. 

In [11]:
# OPTIMIZATION SETUP VARIABLES
lam = 0.5
# define the M so that column i is [1;0] if i is a sire and [0;1] otherwise 
M = np.zeros((2, problem_size))
M[0, sire_indices] = 1
M[1, dam_indices] = 1
# define the right hand side of the constraint Mx = m
m = np.array([[0.5], [0.5]])
# set up the vector for the linear objective
linear_obj = -lam*expected_breeding_values

Finally, we solve the problem using the modules' `solve_qp` function. This utilises Gurobi via an API, a fact which will be important once we start to consider larger problem sizes. 

In [12]:
# SOLVE THE PROBLEM
w_standard = solve_qp(
    P = relationship_matrix,
    q = linear_obj,
    G = None,
    h = None,
    A = M,
    b = m,
    lb = lower_bound,
    ub = upper_bound,
    solver = "gurobi"
)
print(f"QP solution: w = {w_standard}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-26
QP solution: w = [0.5   0.375 0.125]


### Robust Optimization

For robust optimization we adjust the objective function to model the inherent uncertainty in the problem. We may either do this with a quadratic uncertainty set, in which case our objective has an additional square-root term as in
$$
    \min_w \frac{1}{2}w^{T}\Sigma w - \lambda w^{T}\mu - \kappa\sqrt{w^{T}\Omega w}\ \text{ subject to }\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u,
$$
or with a box uncertainty set, in which case our objective has an additional absolute value term as in
 We may either do this with a quadratic uncertainty set, in which case our objective has an additional square-root term as in
$$
    \min_w \frac{1}{2}w^{T}\Sigma w - \lambda w^{T}\mu - \kappa\|\Omega^{\frac{1}{2}} w\|\ \text{ subject to }\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u,
$$
where in both cases $\kappa\in\mathbb{R}$ and $\Omega\in\mathbb{R}^{n\times n}$ are our robust optimization parameters. This is obviously no longer a quadratic problem, so `qpsolvers` is no longer a viable tool. We will instead now need to work with the Gurobi API directly.

In [13]:
# randomly set until received from Gregor 
omega = np.diagflat(np.random.rand(problem_size, 1))
kappa = float(np.random.rand(1))

# a temp(?) workaround to get the square root with SciPy rather than Numpy.
from scipy.linalg import sqrtm
root_omega = sqrtm(omega)

try:
    # create a new model for robust genetic selection
    model = gp.Model("robustGS")

    # define variable of interest as a continuous 
    w = model.addMVar(shape=problem_size, vtype=GRB.CONTINUOUS, name="w")

    # set the objective function
    model.setObjective(
        # STANDARD OPTIMIZATION OBJECTIVE
        0.5*w@(relationship_matrix@w) - lam*w.transpose()@expected_breeding_values,  # standard optimization

        # ROBUST OPTIMIZATION OBJECTIVE WITH QUADRATIC UNCERTAINTY
        # cannot use np.power, get:
        #     TypeError: unsupported operand type(s) for ** or pow(): 'MQuadExpr' and 'float'
        # 0.5*w@(relationship_matrix@w) - lam*w.transpose()@expected_breeding_values - kappa*np.power(np.inner(w, omega@w), 0.5),  # inner product representation
        # 
        # cannot use scipy.linalg.sqrtm and np.nla.norm(ord=2), get:
        #     ValueError: Improper number of dimensions to norm.
        # 0.5*w@(relationship_matrix@w) - lam*w.transpose()@expected_breeding_values - kappa*nla.norm(root_omega@w, ord=2),  # norm representation

        # ROBUST OPTIMIZATION OBJECTIVE WITH BOX UNCERTAINTY
        # cannot use np.avs, get:
        #     TypeError: bad operand type for abs(): 'MQuadExpr'
        # 0.5*w@(relationship_matrix@w) - lam*w.transpose()@expected_breeding_values - kappa*np.abs(np.inner(w, omega@w)),  # robust with box uncertainty
    GRB.MINIMIZE)

    # add sub-to-half constraints
    model.addConstr(M @ w == m, name="sum-to-half")
    # add weight-bound constraints
    model.addConstr(w >= lower_bound, name="lower bound")
    model.addConstr(w <= upper_bound, name="upper bound")

    # solve the problem with Gurobi
    model.optimize()
    print(w.X)
    print(f"Obj: {model.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    # NOTE coding errors sometimes accidentally caught by this 
    print("Encountered an attribute error")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 3 columns and 24 nonzeros
Model fingerprint: 0xd96d3c43
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 2e+00]
  QObjective range [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+00]
Presolve removed 21 rows and 1 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     

In [14]:
# randomly set until received from Gregor
omega = np.diagflat(np.random.rand(problem_size, 1))
kappa = float(np.random.rand(1))

try:
    # create a new model for robust genetic selection
    model = gp.Model("robustGS")

    # define variables of interest as a continuous
    w = model.addMVar(shape=problem_size, vtype=GRB.CONTINUOUS, name="w")
    z = model.addVar(name="z")

    # setup the robust objective function
    model.setObjective(
        0.5*w@(relationship_matrix@w) - lam*w.transpose()@expected_breeding_values -kappa*z,
    GRB.MINIMIZE)

    # add quadratic uncertainty constraint
    model.addConstr(z**2 <= np.inner(w, omega@w), name="uncertainty")
    model.addConstr(z >= 0, name="z positive")
    # add sub-to-half constraints
    model.addConstr(M @ w == m, name="sum-to-half")
    # add weight-bound constraints
    model.addConstr(w >= lower_bound, name="lower bound")
    model.addConstr(w <= upper_bound, name="upper bound")

    # solve the problem with Gurobi
    model.optimize()
    model.printAttr('X')
    print(f"Obj: {model.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    # NOTE coding errors sometimes accidentally caught by this 
    print("Encountered an attribute error")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 23 rows, 4 columns and 25 nonzeros
Model fingerprint: 0xe8d7361e
Model has 3 quadratic objective terms
Model has 3 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [4e-01, 1e+00]
  Objective range  [5e-01, 2e+00]
  QObjective range [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+00]
Presolve removed 22 rows and 1 columns
Presolve time: 0.01s
Presolved: 9 rows, 8 columns, 14 nonzeros
Presolved model has 3 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.200e+01
 Factor NZ  : 4.500e+01
 Factor Ops : 2.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Resid